In [2]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from IPython.display import Markdown, display

# import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import lightgbm as lgb
import itertools
from itertools import product
import datetime
import re
import optuna

import nltk
from nltk.stem.snowball import SnowballStemmer 
from nltk.tokenize import word_tokenize 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.utils import shuffle

import multiprocessing

def printmd(string):
    display(Markdown(string))

# nltk.download('popular')
    
plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 13

# !jt -t chesterish -T -N -kl - make toolbar visible, etc.

In [43]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

scikit-learn==0.23.2
pandas==1.2.0
optuna==2.4.0
numpy==1.19.2
nltk==3.5
matplotlib==3.3.2
lightgbm==2.3.0


Items try find similar items by clustering item names and sales and shops? and maybe more

First time item bein sold (flag)

Maybe add previous month per item transactions? Instead of using the sum of items sold.

Timeseries smoothing

In [3]:
df_items = pd.read_csv('../input/items.csv')
df_item_categories = pd.read_csv('../input/item_categories.csv')
df_sales = pd.read_csv('../input/sales_train.csv')
df_sample_submission = pd.read_csv('../input/sample_submission.csv')
df_shops = pd.read_csv('../input/shops.csv')
df_test_kaggle = pd.read_csv('../input/test.csv')

## Make dataset

In [4]:
columns = ['shop_id', 'item_id', 'date_block_num']

l = []
for date_block_num in sorted(df_sales['date_block_num'].unique()):
    # Add from previous date_block as these are still likely on sale
    pu_shop_id = df_sales[df_sales['date_block_num'] == date_block_num-1]['shop_id'].unique()
    pu_item_id = df_sales[df_sales['date_block_num'] == date_block_num-1]['item_id'].unique()
    
    u_shop_id = df_sales[df_sales['date_block_num'] == date_block_num]['shop_id'].unique()
    u_item_id = df_sales[df_sales['date_block_num'] == date_block_num]['item_id'].unique()
    
#     u_shop_id = np.unique(np.append(u_shop_id, pu_shop_id))
#     u_item_id = np.unique(np.append(u_item_id, pu_item_id))
    l.append(np.array(list(product(u_shop_id, u_item_id, [date_block_num]))))

In [5]:
df = pd.DataFrame(np.vstack(l), columns = columns)
df_train = df_sales[['date_block_num', 'shop_id', 'item_id', 'item_cnt_day']]\
    .groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).sum()
df_train.rename({'item_cnt_day': 'item_cnt_month'}, axis='columns', inplace=True)
df = df.merge(df_train, \
         how='left', \
         left_on=['date_block_num', 'shop_id', 'item_id'], \
         right_on=['date_block_num', 'shop_id', 'item_id'])
df_test_kaggle['date_block_num'] = 34
df = df.append(df_test_kaggle)
df

,shop_id,item_id,date_block_num,item_cnt_month,ID
0,59,22154,0,1.0,NaN
1,59,2552,0,NaN,NaN
2,59,2554,0,NaN,NaN
3,59,2555,0,NaN,NaN
4,59,2564,0,NaN,NaN
...,...,...,...,...,...
214195,45,18454,34,NaN,214195.0
214196,45,16188,34,NaN,214196.0
214197,45,15757,34,NaN,214197.0
214198,45,19648,34,NaN,214198.0


In [6]:
df['item_cnt_month'] = df['item_cnt_month'].clip(0,20).fillna(0)

df['shop_id'] = df['shop_id'].astype(np.uint8)
df['item_id'] = df['item_id'].astype(np.uint16)
df['date_block_num'] = df['date_block_num'].astype(np.uint8)
df['item_cnt_month'] = df['item_cnt_month'].astype(np.float16)

## Simple features

Item category features

In [7]:
f_item_categories = df_items[['item_id', 'item_category_id']]
stemmer_ru = SnowballStemmer("russian")
flatten = lambda t: [item for sublist in t for item in sublist]
temp = df_item_categories.copy()
t = temp['item_category_name'].str.split('-', expand=True)
le = preprocessing.LabelEncoder()
le.fit(t[0].values)
temp['item_category_id_2'] = le.transform(t[0].values)

f_item_categories = f_item_categories.merge(temp[['item_category_id', 'item_category_id_2']], \
                        how='left', \
                        left_on='item_category_id', \
                        right_on='item_category_id')

f_item_categories['item_id'] = f_item_categories['item_id'].astype(np.uint16)
f_item_categories['item_category_id'] = f_item_categories['item_category_id'].astype(np.uint8)
f_item_categories['item_category_id_2'] = f_item_categories['item_category_id_2'].astype(np.uint8)
f_item_categories.head()

,item_id,item_category_id,item_category_id_2
0,0,40,11
1,1,76,15
2,2,40,11
3,3,40,11
4,4,40,11


In [8]:
f_item_categories.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             22170 non-null  uint16
 1   item_category_id    22170 non-null  uint8 
 2   item_category_id_2  22170 non-null  uint8 
dtypes: uint16(1), uint8(2)
memory usage: 259.8 KB


Item text features

In [9]:
# Use stemming to get most commong words in items and then create a matrix of those words where
# 1 signals the existence of the word.
items_processed = df_items.copy()
items_processed['item_name'] = items_processed['item_name'].str.lower()
items_processed['item_name'] = items_processed['item_name'].apply(lambda x: re.sub(r'[^\w]', ' ', x))
items_processed['item_name_tokenized'] = items_processed['item_name'].apply(lambda x: word_tokenize(x))
items_processed['item_name_tokenized_stemmed'] = \
    items_processed['item_name_tokenized']\
    .apply(lambda x: [stemmer_ru.stem(x_item) for x_item in x])

# Get most common stems
top_stemms = pd.Series(\
                       flatten(\
                               items_processed['item_name_tokenized_stemmed'].values))\
                .value_counts()[:50].index
dummy = pd.get_dummies(top_stemms)
dummy_train = pd.DataFrame(index=np.arange(0, len(df_items)), columns=dummy.columns)\
    .fillna(0)\
    .astype(np.bool)

ac = dummy_train.columns
for idx, row in items_processed.iterrows():
    at = np.array(row['item_name_tokenized_stemmed'])
    dummy_train.loc[idx, ac[np.where(np.isin(ac, at))[0]]] = 1
    
f_item_name = dummy_train.reset_index().rename({'index': 'item_id'}, axis='columns')
f_item_name['item_id'] = f_item_name['item_id'].astype(np.uint16)
f_item_name.head()

,item_id,1,1с,2,2cd,3,360,3d,4,5,...,набор,регион,русск,с,сб,субтитр,фигурк,фирм,футболк,цифров
0,0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2,2,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
f_item_name = f_item_name.astype({ x: np.bool for x in np.delete(f_item_name.columns, 0)})

In [11]:
f_item_name.head()

,item_id,1,1с,2,2cd,3,360,3d,4,5,...,набор,регион,русск,с,сб,субтитр,фигурк,фирм,футболк,цифров
0,0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,2,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Simple shop features

In [12]:
temp = df_shops.copy()
t = df_shops['shop_name'].str.split(' ', expand=True)
le = preprocessing.LabelEncoder()
t0 = t[0].replace('!', '', regex=True).values
le.fit(t0)
temp['shop_city_id'] = le.transform(t0)
temp['shopping_center_ТЦ'] = temp['shop_name'].str.contains('ТЦ')*1
temp['mall_ТРЦ'] = temp['shop_name'].str.contains('ТРЦ')*1
f_shop_id = temp[['shop_id', 'shop_city_id', 'shopping_center_ТЦ', 'mall_ТРЦ']].copy()
f_shop_id.head()

,shop_id,shop_city_id,shopping_center_ТЦ,mall_ТРЦ
0,0,29,0,0
1,1,29,1,0
2,2,0,1,0
3,3,1,0,0
4,4,2,1,0


In [13]:
f_shop_id['shop_id'] = f_shop_id['shop_id'].astype(np.uint8)
f_shop_id['shop_city_id'] = f_shop_id['shop_city_id'].astype(np.uint8)
f_shop_id['shopping_center_ТЦ'] = f_shop_id['shopping_center_ТЦ'].astype(np.uint8)
f_shop_id['mall_ТРЦ'] = f_shop_id['mall_ТРЦ'].astype(np.uint8)

In [14]:
# In both cases the shops seems to have changed locations after date_block_num 1
# Not sure if it makes sense to change the shop id or not
print(df_sales[df_sales['shop_id'].isin([0, 57])].groupby('date_block_num').agg({'shop_id': 'mean'}).head())
print(df_sales[df_sales['shop_id'].isin([1, 58])].groupby('date_block_num').agg({'shop_id': 'mean'}).head())

                shop_id
date_block_num         
0                     0
1                     0
2                    57
3                    57
4                    57
                shop_id
date_block_num         
0                     1
1                     1
2                    58
3                    58
4                    58


## Mean-encoding

In [15]:
mean_encoding_lag = [x for x in range(1, 4)]
mean_encoding_lag

[1, 2, 3]

shop_id and item_id mean-encoding

In [16]:
shop_item_mean_encoding = df[['shop_id', 'item_id', 'date_block_num', 'item_cnt_month']].copy()
f_shop_item_mean_encoding = shop_item_mean_encoding[['shop_id', 'item_id', 'date_block_num']].copy()
for i in mean_encoding_lag:
    t = shop_item_mean_encoding.copy()
    t['date_block_num'] = t['date_block_num'] + i
    t.rename({'item_cnt_month': 'shop_item_cnt_month_lag_'+str(i)}, axis='columns', inplace=True)
    f_shop_item_mean_encoding = f_shop_item_mean_encoding.merge(t, how='left', \
                                    left_on=['shop_id', 'item_id', 'date_block_num'],
                                    right_on=['shop_id', 'item_id', 'date_block_num'])
f_shop_item_mean_encoding.fillna(0, inplace=True)
f_shop_item_mean_encoding.head()

,shop_id,item_id,date_block_num,shop_item_cnt_month_lag_1,shop_item_cnt_month_lag_2,shop_item_cnt_month_lag_3
0,59,22154,0,0.0,0.0,0.0
1,59,2552,0,0.0,0.0,0.0
2,59,2554,0,0.0,0.0,0.0
3,59,2555,0,0.0,0.0,0.0
4,59,2564,0,0.0,0.0,0.0


shop_id mean-encoding

In [17]:
# Add shop_cnt to each shop_id, date_block_num combo.
shop_mean_encoding = df[['shop_id', 'date_block_num', 'item_cnt_month']].copy()
shop_mean_encoding = shop_mean_encoding.groupby(['shop_id', 'date_block_num'], as_index=False).mean()
f_shop_mean_encoding = shop_mean_encoding[['shop_id', 'date_block_num']]

for i in mean_encoding_lag:
    t = shop_mean_encoding.copy()
    t['date_block_num'] = t['date_block_num'] + i
    t.rename({'item_cnt_month': 'shop_cnt_month_lag_'+str(i)}, axis='columns', inplace=True)
    f_shop_mean_encoding = f_shop_mean_encoding.merge(t, how='left', \
                                left_on=['shop_id', 'date_block_num'],
                                right_on=['shop_id', 'date_block_num'])
f_shop_mean_encoding.fillna(0, inplace=True)
f_shop_mean_encoding['shop_id'] = f_shop_mean_encoding['shop_id'].astype(np.uint8)
f_shop_mean_encoding['date_block_num'] = f_shop_mean_encoding['date_block_num'].astype(np.uint8)
f_shop_mean_encoding.head()

,shop_id,date_block_num,shop_cnt_month_lag_1,shop_cnt_month_lag_2,shop_cnt_month_lag_3
0,0,0,0.000000,0.0,0.0
1,0,1,0.666992,0.0,0.0
2,1,0,0.000000,0.0,0.0
3,1,1,0.361328,0.0,0.0
4,2,0,0.000000,0.0,0.0


shop_city_id, shopping_center_ТЦ, mall_ТРЦ mean-encoding

In [18]:
shop_custom_mean_encoding = df.merge(f_shop_id, how='left', \
         left_on='shop_id', right_on='shop_id')
shop_city_id_mean_encoding = shop_custom_mean_encoding\
    [['date_block_num', 'shop_city_id', 'item_cnt_month']]\
    .groupby(['date_block_num', 'shop_city_id'], as_index=False)\
    .mean()
shopping_center_mean_encoding = shop_custom_mean_encoding\
    [['date_block_num', 'shopping_center_ТЦ', 'item_cnt_month']]\
    .groupby(['date_block_num', 'shopping_center_ТЦ'], as_index=False)\
    .mean()
mall_mean_encoding = shop_custom_mean_encoding\
    [['date_block_num', 'mall_ТРЦ', 'item_cnt_month']]\
    .groupby(['date_block_num', 'mall_ТРЦ'], as_index=False)\
    .mean()

In [19]:
f_shop_city_id_mean_encoding = shop_city_id_mean_encoding.copy().drop('item_cnt_month', axis='columns')
f_shopping_center_mean_encoding = shopping_center_mean_encoding.copy().drop('item_cnt_month', axis='columns')
f_mall_mean_encoding = mall_mean_encoding.copy().drop('item_cnt_month', axis='columns')
for i in mean_encoding_lag:
    t1 = shop_city_id_mean_encoding.copy()
    t1['date_block_num'] = t1['date_block_num'] + i
    f_shop_city_id_mean_encoding = f_shop_city_id_mean_encoding.merge(t1, how='left', \
                                       left_on=['date_block_num', 'shop_city_id'], \
                                       right_on=['date_block_num', 'shop_city_id'])
    f_shop_city_id_mean_encoding.rename({'item_cnt_month': 'shop_city_id_item_cnt_month_lag_'+str(i)}, \
                                        axis='columns', inplace=True)
    
    t2 = shopping_center_mean_encoding.copy()
    t2['date_block_num'] = t2['date_block_num'] + i
    f_shopping_center_mean_encoding = f_shopping_center_mean_encoding.merge(t2, how='left', \
                                       left_on=['date_block_num', 'shopping_center_ТЦ'], \
                                       right_on=['date_block_num', 'shopping_center_ТЦ'])
    f_shopping_center_mean_encoding.rename({'item_cnt_month': 'shopping_center_item_cnt_month_lag_'+str(i)}, \
                                        axis='columns', inplace=True)
    
    t3 = mall_mean_encoding.copy()
    t3['date_block_num'] = t3['date_block_num'] + i
    f_mall_mean_encoding = f_mall_mean_encoding.merge(t3, how='left', \
                                       left_on=['date_block_num', 'mall_ТРЦ'], \
                                       right_on=['date_block_num', 'mall_ТРЦ'])
    f_mall_mean_encoding.rename({'item_cnt_month': 'mall_item_cnt_month_lag_'+str(i)}, \
                                        axis='columns', inplace=True)

f_shop_city_id_mean_encoding['date_block_num'] = \
    f_shop_city_id_mean_encoding['date_block_num']\
    .astype(np.uint8)
f_shop_city_id_mean_encoding['shop_city_id'] = \
    f_shop_city_id_mean_encoding['shop_city_id']\
    .astype(np.uint8)
f_shopping_center_mean_encoding['date_block_num'] = \
    f_shopping_center_mean_encoding['date_block_num']\
    .astype(np.uint8)
f_shopping_center_mean_encoding['shopping_center_ТЦ'] = \
    f_shopping_center_mean_encoding['shopping_center_ТЦ']\
    .astype(np.uint8)
f_mall_mean_encoding['date_block_num'] = \
    f_mall_mean_encoding['date_block_num']\
    .astype(np.uint8)
f_mall_mean_encoding['mall_ТРЦ'] = \
    f_mall_mean_encoding['mall_ТРЦ']\
    .astype(np.uint8)

In [20]:
f_shop_city_id_mean_encoding.head()

,date_block_num,shop_city_id,shop_city_id_item_cnt_month_lag_1,shop_city_id_item_cnt_month_lag_2,shop_city_id_item_cnt_month_lag_3
0,0,0,NaN,NaN,NaN
1,0,1,NaN,NaN,NaN
2,0,2,NaN,NaN,NaN
3,0,4,NaN,NaN,NaN
4,0,6,NaN,NaN,NaN


In [21]:
f_shopping_center_mean_encoding.head()

,date_block_num,shopping_center_ТЦ,shopping_center_item_cnt_month_lag_1,shopping_center_item_cnt_month_lag_2,shopping_center_item_cnt_month_lag_3
0,0,0,NaN,NaN,NaN
1,0,1,NaN,NaN,NaN
2,1,0,0.307861,NaN,NaN
3,1,1,0.375732,NaN,NaN
4,2,0,0.297363,0.307861,NaN


In [22]:
f_mall_mean_encoding.head()

,date_block_num,mall_ТРЦ,mall_item_cnt_month_lag_1,mall_item_cnt_month_lag_2,mall_item_cnt_month_lag_3
0,0,0,NaN,NaN,NaN
1,0,1,NaN,NaN,NaN
2,1,0,0.361084,NaN,NaN
3,1,1,0.272217,NaN,NaN
4,2,0,0.342529,0.361084,NaN


item_id mean-encoding

In [23]:
# Add shop_cnt to each shop_id, date_block_num combo.
item_mean_encoding = df[['item_id', 'date_block_num', 'item_cnt_month']].copy()
item_mean_encoding = item_mean_encoding.groupby(['item_id', 'date_block_num'], as_index=False).mean()
f_item_mean_encoding = item_mean_encoding[['item_id', 'date_block_num']]

for i in mean_encoding_lag:
    t = item_mean_encoding.copy()
    t['date_block_num'] = t['date_block_num'] + i
    t.rename({'item_cnt_month': 'item_cnt_month_lag_'+str(i)}, axis='columns', inplace=True)
    f_item_mean_encoding = f_item_mean_encoding.merge(t, how='left', \
                                left_on=['item_id', 'date_block_num'],
                                right_on=['item_id', 'date_block_num'])
f_item_mean_encoding.fillna(0, inplace=True)
f_item_mean_encoding['item_id'] = f_item_mean_encoding['item_id'].astype(np.uint16)
f_item_mean_encoding['date_block_num'] = f_item_mean_encoding['date_block_num'].astype(np.uint8)
f_item_mean_encoding.head()

,item_id,date_block_num,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3
0,0,20,0.000000,0.000000,0.000000
1,1,15,0.000000,0.000000,0.000000
2,1,18,0.000000,0.000000,0.040802
3,1,19,0.020004,0.000000,0.000000
4,1,20,0.019608,0.020004,0.000000


item_category mean-encoding

In [24]:
item_cat_mean_enc = df_train.merge(f_item_categories, how='left', \
                                             left_on=['item_id'], right_on=['item_id'])
item_cat_mean_enc = \
    item_cat_mean_enc[['item_category_id', 'date_block_num', 'item_cnt_month']]
item_cat_mean_enc = \
    item_cat_mean_enc.groupby(['item_category_id', 'date_block_num'], as_index=False).mean()
f_item_category_id = item_cat_mean_enc[['item_category_id', 'date_block_num']].copy()
item_cat_mean_enc['item_cnt_month'] = item_cat_mean_enc['item_cnt_month'].astype(np.float16)
for i in mean_encoding_lag:
    t = item_cat_mean_enc.copy()
    t['date_block_num'] = t['date_block_num'] + i
    t.rename({'item_cnt_month': 'item_category_id_cnt_month_lag_'+str(i)}, axis='columns', inplace=True)
    f_item_category_id = f_item_category_id.merge(t, how='left', \
                            left_on=['item_category_id', 'date_block_num'],
                            right_on=['item_category_id', 'date_block_num'])
f_item_category_id.fillna(0, inplace=True)
f_item_category_id['item_category_id'] = f_item_category_id['item_category_id'].astype(np.uint8)
f_item_category_id['date_block_num'] = f_item_category_id['date_block_num'].astype(np.uint8)
f_item_category_id.head()

,item_category_id,date_block_num,item_category_id_cnt_month_lag_1,item_category_id_cnt_month_lag_2,item_category_id_cnt_month_lag_3
0,0,0,0.0,0.0,0.0
1,0,1,1.0,0.0,0.0
2,0,3,0.0,1.0,1.0
3,1,0,0.0,0.0,0.0
4,1,1,1.0,0.0,0.0


item_category_2 mean-encoding

In [25]:
item_cat2_mean_enc = df_train.merge(f_item_categories, how='left', \
                                             left_on=['item_id'], right_on=['item_id'])
item_cat2_mean_enc = \
    item_cat2_mean_enc[['item_category_id_2', 'date_block_num', 'item_cnt_month']]
item_cat2_mean_enc = \
    item_cat2_mean_enc.groupby(['item_category_id_2', 'date_block_num'], as_index=False).mean()
f_item_category_id2 = item_cat2_mean_enc[['item_category_id_2', 'date_block_num']].copy()
item_cat2_mean_enc['item_cnt_month'] = item_cat2_mean_enc['item_cnt_month'].astype(np.float16)
for i in mean_encoding_lag:
    t = item_cat2_mean_enc.copy()
    t['date_block_num'] = t['date_block_num'] + i
    t.rename({'item_cnt_month': 'item_category_id2_cnt_month_lag_'+str(i)}, axis='columns', inplace=True)
    f_item_category_id2 = f_item_category_id2.merge(t, how='left', \
                            left_on=['item_category_id_2', 'date_block_num'],
                            right_on=['item_category_id_2', 'date_block_num'])
f_item_category_id2.fillna(0, inplace=True)
f_item_category_id2['item_category_id_2'] = f_item_category_id2['item_category_id_2'].astype(np.uint8)
f_item_category_id2['date_block_num'] = f_item_category_id2['date_block_num'].astype(np.uint8)
f_item_category_id2.head()

,item_category_id_2,date_block_num,item_category_id2_cnt_month_lag_1,item_category_id2_cnt_month_lag_2,item_category_id2_cnt_month_lag_3
0,0,0,0.000000,0.0,0.0
1,0,1,1.000000,0.0,0.0
2,0,3,0.000000,1.0,1.0
3,1,0,0.000000,0.0,0.0
4,1,1,2.566406,0.0,0.0


Add item first occurance

In [26]:
temp = df.copy()
temp['shop_item'] = temp['shop_id'].astype(str) + '_' + temp['item_id'].astype(str)
columns_item = ['date_block_num', 'item_id']
columns_shop_item = ['date_block_num', 'shop_item']

exiting_item = set([])
existing_shop_item = set([])

first_item = []
first_shop_item = []

for date_block_num in np.sort(temp['date_block_num'].unique()):
    t = temp[temp['date_block_num'] == date_block_num]
    unique_item = set(t['item_id'].unique())
    unique_shop_item = set(t['shop_item'].unique())
    
    diff_item = unique_item - exiting_item
    diff_shop_item = unique_shop_item - existing_shop_item
    
    first_item.append(list(product([date_block_num], list(diff_item))))
    first_shop_item.append(list(product([date_block_num], list(diff_shop_item))))
    
    exiting_item.update(unique_item)
    existing_shop_item.update(diff_shop_item)

f_first_item = pd.DataFrame(np.vstack(first_item), columns=columns_item)
f_first_shop_item = pd.DataFrame(np.vstack(first_shop_item), columns=columns_shop_item)

# Split shop_item into shop_id and item_id
temp = f_first_shop_item['shop_item']\
    .str.split('_', expand=True)\
    .rename({0: 'shop_id', 1:'item_id'}, axis='columns')
f_first_shop_item = f_first_shop_item.merge(temp, how='left', left_index=True, right_index=True)
f_first_shop_item.drop(['shop_item'], axis='columns', inplace=True)

f_first_item['first_item_occurance'] = 1
f_first_shop_item['first_shop_item_occurance'] = 1

f_first_item['date_block_num'] = f_first_item['date_block_num'].astype(np.uint8)
f_first_item['item_id'] = f_first_item['item_id'].astype(np.uint16)

f_first_shop_item['date_block_num'] = f_first_shop_item['date_block_num'].astype(np.uint8)
f_first_shop_item['shop_id'] = f_first_shop_item['shop_id'].astype(np.uint8)
f_first_shop_item['item_id'] = f_first_shop_item['item_id'].astype(np.uint16)

Data leakage could not be detected

## Join generated features

In [29]:
df_input = df.copy()
df_input = df_input.merge(f_item_categories, how='left', left_on=['item_id'], right_on=['item_id'])
df_input = df_input.merge(f_item_name, how='left', left_on=['item_id'], right_on=['item_id'])
df_input = df_input.merge(f_shop_id, how='left', left_on=['shop_id'], right_on=['shop_id'])
df_input = df_input.merge(f_shop_item_mean_encoding, how='left', \
                          left_on=['date_block_num', 'shop_id', 'item_id'], \
                          right_on=['date_block_num', 'shop_id', 'item_id'])
df_input = df_input.merge(f_shop_mean_encoding, how='left', \
                          left_on=['date_block_num', 'shop_id'], right_on=['date_block_num', 'shop_id'])
df_input = df_input.merge(f_shop_city_id_mean_encoding, how='left', \
                          left_on=['date_block_num', 'shop_city_id'], \
                          right_on=['date_block_num', 'shop_city_id'])
df_input = df_input.merge(f_shopping_center_mean_encoding, how='left', \
                          left_on=['date_block_num', 'shopping_center_ТЦ'], \
                          right_on=['date_block_num', 'shopping_center_ТЦ'])
df_input = df_input.merge(f_mall_mean_encoding, how='left', \
                          left_on=['date_block_num', 'mall_ТРЦ'], \
                          right_on=['date_block_num', 'mall_ТРЦ'])
df_input = df_input.merge(f_item_mean_encoding, how='left', \
                          left_on=['date_block_num', 'item_id'], \
                          right_on=['date_block_num', 'item_id'])
df_input = df_input.merge(f_item_category_id, how='left', \
                          left_on=['date_block_num', 'item_category_id'], \
                          right_on=['date_block_num', 'item_category_id'])
df_input = df_input.merge(f_item_category_id2, how='left', \
                          left_on=['date_block_num', 'item_category_id_2'], \
                          right_on=['date_block_num', 'item_category_id_2'])
df_input = df_input.merge(f_first_item, how='left', \
                          left_on=['date_block_num', 'item_id'], \
                          right_on=['date_block_num', 'item_id'])
df_input = df_input.merge(f_first_shop_item, how='left', \
                          left_on=['date_block_num', 'shop_id', 'item_id'], \
                          right_on=['date_block_num', 'shop_id', 'item_id'])
df_input.loc[df_input['first_item_occurance'].isna().values, \
                   'first_item_occurance'] = 0
df_input['first_item_occurance'] = df_input['first_item_occurance'].astype(np.bool)
df_input.loc[df_input['first_shop_item_occurance'].isna().values, \
                   'first_shop_item_occurance'] = 0
df_input['first_shop_item_occurance'] = df_input['first_shop_item_occurance'].astype(np.bool)

## Prepare data for model

In [32]:
df_input = df_input.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', 'col', x))

## Train model

Find optimal hyperparameters

In [ ]:
# Without pu_shop_id and pu_item_id
def objective(trial):

    # 2. Suggest values of the hyperparameters using a trial object.
    param = {
        'objective': 'mse',
        'metric': 'root_mean_squared_error',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    gbm = lgb.train(param, train_lgb, valid_sets=val_lgb)
    preds = gbm.predict(df_val_input.drop(['item_cnt_month', 'ID'], axis='columns').values)
    msre = mean_squared_error(df_val_input['item_cnt_month'], preds, squared=False)
    return msre

df_train_input = df_input[df_input['date_block_num']<33]
df_val_input = df_input[df_input['date_block_num']==33]
df_test_input = df_input[df_input['date_block_num']==34]

train_lgb = lgb.Dataset(df_train_input.drop(['item_cnt_month', 'ID'], axis='columns'), \
            label=df_train_input['item_cnt_month'])
val_lgb = lgb.Dataset(df_val_input.drop(['item_cnt_month', 'ID'], axis='columns'), \
            label=df_val_input['item_cnt_month'])

param = {'num_leaves': 200, \
         'objective': 'mse', \
         'metric': 'root_mean_squared_error',\
         'verbosity': 0}

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=250)

[I 2021-01-27 01:36:51,136] A new study created in memory with name: no-name-8f1a13ea-de29-4ed1-bbd9-f1ffef63407c


[1]	valid_0's rmse: 1.10634
[2]	valid_0's rmse: 1.08474
[3]	valid_0's rmse: 1.05763
[4]	valid_0's rmse: 1.03377
[5]	valid_0's rmse: 1.01614
[6]	valid_0's rmse: 0.996928
[7]	valid_0's rmse: 0.983816
[8]	valid_0's rmse: 0.974788
[9]	valid_0's rmse: 0.966373
[10]	valid_0's rmse: 0.95406
[11]	valid_0's rmse: 0.944294
[12]	valid_0's rmse: 0.935231
[13]	valid_0's rmse: 0.927189
[14]	valid_0's rmse: 0.923016
[15]	valid_0's rmse: 0.921177
[16]	valid_0's rmse: 0.917563
[17]	valid_0's rmse: 0.915145
[18]	valid_0's rmse: 0.912683
[19]	valid_0's rmse: 0.910155
[20]	valid_0's rmse: 0.905895
[21]	valid_0's rmse: 0.904751
[22]	valid_0's rmse: 0.902445
[23]	valid_0's rmse: 0.901475
[24]	valid_0's rmse: 0.899775
[25]	valid_0's rmse: 0.898431
[26]	valid_0's rmse: 0.897908
[27]	valid_0's rmse: 0.896376
[28]	valid_0's rmse: 0.895626
[29]	valid_0's rmse: 0.894977
[30]	valid_0's rmse: 0.895245
[31]	valid_0's rmse: 0.894133
[32]	valid_0's rmse: 0.894325
[33]	valid_0's rmse: 0.894328
[34]	valid_0's rmse: 0.89

[I 2021-01-27 01:39:02,060] Trial 0 finished with value: 0.8906617204131841 and parameters: {'lambda_l1': 1.6610668148085814e-06, 'lambda_l2': 0.0020858609664307533, 'num_leaves': 123, 'feature_fraction': 0.4941836798690168, 'bagging_fraction': 0.724273828327328, 'bagging_freq': 2, 'min_child_samples': 67}. Best is trial 0 with value: 0.8906617204131841.


[1]	valid_0's rmse: 1.11357
[2]	valid_0's rmse: 1.09374
[3]	valid_0's rmse: 1.06914
[4]	valid_0's rmse: 1.05231
[5]	valid_0's rmse: 1.03514
[6]	valid_0's rmse: 1.0197
[7]	valid_0's rmse: 1.0057
[8]	valid_0's rmse: 0.995266
[9]	valid_0's rmse: 0.989465
[10]	valid_0's rmse: 0.975793
[11]	valid_0's rmse: 0.965397
[12]	valid_0's rmse: 0.95651
[13]	valid_0's rmse: 0.949382
[14]	valid_0's rmse: 0.945088
[15]	valid_0's rmse: 0.941791
[16]	valid_0's rmse: 0.93856
[17]	valid_0's rmse: 0.935071
[18]	valid_0's rmse: 0.93258
[19]	valid_0's rmse: 0.927754
[20]	valid_0's rmse: 0.923685
[21]	valid_0's rmse: 0.921806
[22]	valid_0's rmse: 0.919084
[23]	valid_0's rmse: 0.918309
[24]	valid_0's rmse: 0.916435
[25]	valid_0's rmse: 0.914971
[26]	valid_0's rmse: 0.914415
[27]	valid_0's rmse: 0.911882
[28]	valid_0's rmse: 0.911199
[29]	valid_0's rmse: 0.910267
[30]	valid_0's rmse: 0.909472
[31]	valid_0's rmse: 0.908807
[32]	valid_0's rmse: 0.90821
[33]	valid_0's rmse: 0.90809
[34]	valid_0's rmse: 0.907337
[35

[I 2021-01-27 01:39:46,468] Trial 1 finished with value: 0.9030632728214903 and parameters: {'lambda_l1': 1.1323287688505725e-08, 'lambda_l2': 5.155357953844033e-05, 'num_leaves': 37, 'feature_fraction': 0.45972958923190654, 'bagging_fraction': 0.8926443738214717, 'bagging_freq': 4, 'min_child_samples': 74}. Best is trial 0 with value: 0.8906617204131841.


...

[1]	valid_0's rmse: 1.09726
[2]	valid_0's rmse: 1.06438
[3]	valid_0's rmse: 1.03574
[4]	valid_0's rmse: 1.0123
[5]	valid_0's rmse: 0.993893
[6]	valid_0's rmse: 0.975596
[7]	valid_0's rmse: 0.962447
[8]	valid_0's rmse: 0.950813
[9]	valid_0's rmse: 0.938477
[10]	valid_0's rmse: 0.930639
[11]	valid_0's rmse: 0.925279
[12]	valid_0's rmse: 0.919794
[13]	valid_0's rmse: 0.91364
[14]	valid_0's rmse: 0.910898
[15]	valid_0's rmse: 0.908261
[16]	valid_0's rmse: 0.905651
[17]	valid_0's rmse: 0.904156
[18]	valid_0's rmse: 0.902074
[19]	valid_0's rmse: 0.901069
[20]	valid_0's rmse: 0.900231
[21]	valid_0's rmse: 0.899474
[22]	valid_0's rmse: 0.898685
[23]	valid_0's rmse: 0.896776
[24]	valid_0's rmse: 0.896194
[25]	valid_0's rmse: 0.895452
[26]	valid_0's rmse: 0.895224
[27]	valid_0's rmse: 0.895141
[28]	valid_0's rmse: 0.89675
[29]	valid_0's rmse: 0.896334
[30]	valid_0's rmse: 0.896151
[31]	valid_0's rmse: 0.896254
[32]	valid_0's rmse: 0.895987
[33]	valid_0's rmse: 0.896116
[34]	valid_0's rmse: 0.896

[I 2021-01-27 04:11:51,010] Trial 206 finished with value: 0.8932385855495425 and parameters: {'lambda_l1': 0.007684982503893611, 'lambda_l2': 0.01795191010109123, 'num_leaves': 92, 'feature_fraction': 0.9432651119342407, 'bagging_fraction': 0.46808926879579005, 'bagging_freq': 7, 'min_child_samples': 94}. Best is trial 191 with value: 0.8705470858980957.


[1]	valid_0's rmse: 1.0942
[2]	valid_0's rmse: 1.06016
[3]	valid_0's rmse: 1.03193
[4]	valid_0's rmse: 1.00765
[5]	valid_0's rmse: 0.988749
[6]	valid_0's rmse: 0.971023
[7]	valid_0's rmse: 0.9574
[8]	valid_0's rmse: 0.945609
[9]	valid_0's rmse: 0.935295
[10]	valid_0's rmse: 0.925079
[11]	valid_0's rmse: 0.917635
[12]	valid_0's rmse: 0.913634
[13]	valid_0's rmse: 0.909215
[14]	valid_0's rmse: 0.906556
[15]	valid_0's rmse: 0.902105
[16]	valid_0's rmse: 0.900195
[17]	valid_0's rmse: 0.898723
[18]	valid_0's rmse: 0.896672
[19]	valid_0's rmse: 0.895944
[20]	valid_0's rmse: 0.894986
[21]	valid_0's rmse: 0.894223
[22]	valid_0's rmse: 0.893372
[23]	valid_0's rmse: 0.892744
[24]	valid_0's rmse: 0.892474
[25]	valid_0's rmse: 0.892092
[26]	valid_0's rmse: 0.891221
[27]	valid_0's rmse: 0.890753
[28]	valid_0's rmse: 0.890972
[29]	valid_0's rmse: 0.890618
[30]	valid_0's rmse: 0.890344
[31]	valid_0's rmse: 0.890055
[32]	valid_0's rmse: 0.890064
[33]	valid_0's rmse: 0.889974
[34]	valid_0's rmse: 0.889

[I 2021-01-27 04:12:32,622] Trial 207 finished with value: 0.8908463401301129 and parameters: {'lambda_l1': 0.0021089186344895212, 'lambda_l2': 0.03735328792408073, 'num_leaves': 121, 'feature_fraction': 0.9254884044514757, 'bagging_fraction': 0.5879129645016614, 'bagging_freq': 7, 'min_child_samples': 91}. Best is trial 191 with value: 0.8705470858980957.


[1]	valid_0's rmse: 1.09542
[2]	valid_0's rmse: 1.06298
[3]	valid_0's rmse: 1.03413
[4]	valid_0's rmse: 1.0112
[5]	valid_0's rmse: 0.992619
[6]	valid_0's rmse: 0.973976
[7]	valid_0's rmse: 0.960021
[8]	valid_0's rmse: 0.947749
[9]	valid_0's rmse: 0.93917
[10]	valid_0's rmse: 0.931782
[11]	valid_0's rmse: 0.924882
[12]	valid_0's rmse: 0.92047
[13]	valid_0's rmse: 0.915703
[14]	valid_0's rmse: 0.912669
[15]	valid_0's rmse: 0.909123
[16]	valid_0's rmse: 0.906782
[17]	valid_0's rmse: 0.904913
[18]	valid_0's rmse: 0.903274
[19]	valid_0's rmse: 0.901841
[20]	valid_0's rmse: 0.901404
[21]	valid_0's rmse: 0.900917
[22]	valid_0's rmse: 0.899651
[23]	valid_0's rmse: 0.89889
[24]	valid_0's rmse: 0.898119
[25]	valid_0's rmse: 0.897637
[26]	valid_0's rmse: 0.897456
[27]	valid_0's rmse: 0.897009
[28]	valid_0's rmse: 0.896762
[29]	valid_0's rmse: 0.89629
[30]	valid_0's rmse: 0.896668
[31]	valid_0's rmse: 0.895355
[32]	valid_0's rmse: 0.895372
[33]	valid_0's rmse: 0.895564
[34]	valid_0's rmse: 0.89538

In [ ]:
# results of hyper param optimization
{'lambda_l1': 0.0027195002541221284, \
'lambda_l2': 0.006749781970807508, \
'num_leaves': 118, \
'feature_fraction': 0.9347938981568078, \
'bagging_fraction': 0.5923021997008711, \
'bagging_freq': 7, \
'min_child_samples': 93}

Ensemble (holdout averaging)

In [37]:
num_round = 1500
param = {'num_leaves': 200, 'objective': 'mse', 'metric': 'root_mean_squared_error'}
df_test_input = df_input[df_input['date_block_num']==34]
res = {}
for date_block_cutoff in ([29, 30, 31, 32, 33]):
    df_train_input = df_input[df_input['date_block_num'] < date_block_cutoff]
    df_val_input = df_input[df_input['date_block_num']==date_block_cutoff]
    
    train_lgb = lgb.Dataset(df_train_input.drop(['item_cnt_month', 'ID'], axis='columns'), \
                label=df_train_input['item_cnt_month'])
    val_lgb = lgb.Dataset(df_val_input.drop(['item_cnt_month', 'ID'], axis='columns'), \
                label=df_val_input['item_cnt_month'])
    bst = lgb.train(param, \
                    train_lgb, \
                    num_round, \
                    valid_sets=[train_lgb, val_lgb], \
                    callbacks=[lgb.early_stopping(100)])
    
    pred_test = bst.predict(df_test_input.drop(['item_cnt_month', 'ID'], axis='columns').values)
    sub = df_test_input[['ID']].copy()
    sub['ID'] = sub['ID'].astype(np.int32)
    sub['item_cnt_month'] = pred_test
    res[date_block_cutoff] = sub

[1]	training's rmse: 1.08234	valid_1's rmse: 0.914522
Training until validation scores don't improve for 100 rounds
[2]	training's rmse: 1.04016	valid_1's rmse: 0.878278
[3]	training's rmse: 1.00406	valid_1's rmse: 0.849007
[4]	training's rmse: 0.973073	valid_1's rmse: 0.826028
[5]	training's rmse: 0.946982	valid_1's rmse: 0.807903
[6]	training's rmse: 0.924826	valid_1's rmse: 0.792309
[7]	training's rmse: 0.905527	valid_1's rmse: 0.778277
[8]	training's rmse: 0.888974	valid_1's rmse: 0.769471
[9]	training's rmse: 0.874974	valid_1's rmse: 0.760646
[10]	training's rmse: 0.862593	valid_1's rmse: 0.755044
[11]	training's rmse: 0.852315	valid_1's rmse: 0.750485
[12]	training's rmse: 0.843192	valid_1's rmse: 0.745495
[13]	training's rmse: 0.835186	valid_1's rmse: 0.74111
[14]	training's rmse: 0.828359	valid_1's rmse: 0.737493
[15]	training's rmse: 0.822307	valid_1's rmse: 0.735584
[16]	training's rmse: 0.816777	valid_1's rmse: 0.733703
[17]	training's rmse: 0.811721	valid_1's rmse: 0.733611

[127]	training's rmse: 0.699082	valid_1's rmse: 0.651803
[128]	training's rmse: 0.698868	valid_1's rmse: 0.651735
[129]	training's rmse: 0.698569	valid_1's rmse: 0.651585
[130]	training's rmse: 0.698332	valid_1's rmse: 0.651544
[131]	training's rmse: 0.698108	valid_1's rmse: 0.651704
[132]	training's rmse: 0.697862	valid_1's rmse: 0.651622
[133]	training's rmse: 0.697356	valid_1's rmse: 0.651636
[134]	training's rmse: 0.696843	valid_1's rmse: 0.651659
[135]	training's rmse: 0.696548	valid_1's rmse: 0.651722
[136]	training's rmse: 0.696289	valid_1's rmse: 0.65166
[137]	training's rmse: 0.695804	valid_1's rmse: 0.651913
[138]	training's rmse: 0.695555	valid_1's rmse: 0.651977
[139]	training's rmse: 0.695323	valid_1's rmse: 0.651922
[140]	training's rmse: 0.695104	valid_1's rmse: 0.652216
[141]	training's rmse: 0.694812	valid_1's rmse: 0.652224
[142]	training's rmse: 0.694502	valid_1's rmse: 0.65231
[143]	training's rmse: 0.694266	valid_1's rmse: 0.652276
[144]	training's rmse: 0.69405	va

[201]	training's rmse: 0.675219	valid_1's rmse: 0.698112
[202]	training's rmse: 0.675064	valid_1's rmse: 0.698112
[203]	training's rmse: 0.674903	valid_1's rmse: 0.698151
[204]	training's rmse: 0.674764	valid_1's rmse: 0.6981
[205]	training's rmse: 0.674588	valid_1's rmse: 0.698102
[206]	training's rmse: 0.674292	valid_1's rmse: 0.698109
[207]	training's rmse: 0.674105	valid_1's rmse: 0.698184
[208]	training's rmse: 0.673989	valid_1's rmse: 0.698186
[209]	training's rmse: 0.673819	valid_1's rmse: 0.698211
[210]	training's rmse: 0.673542	valid_1's rmse: 0.698202
[211]	training's rmse: 0.673211	valid_1's rmse: 0.698371
[212]	training's rmse: 0.672878	valid_1's rmse: 0.698206
[213]	training's rmse: 0.672759	valid_1's rmse: 0.698214
[214]	training's rmse: 0.672613	valid_1's rmse: 0.698186
[215]	training's rmse: 0.672503	valid_1's rmse: 0.698201
[216]	training's rmse: 0.672346	valid_1's rmse: 0.698304
[217]	training's rmse: 0.67205	valid_1's rmse: 0.698304
[218]	training's rmse: 0.671663	va

[194]	training's rmse: 0.677779	valid_1's rmse: 0.782264
[195]	training's rmse: 0.677502	valid_1's rmse: 0.782343
[196]	training's rmse: 0.677327	valid_1's rmse: 0.782266
[197]	training's rmse: 0.677155	valid_1's rmse: 0.782343
[198]	training's rmse: 0.676307	valid_1's rmse: 0.782291
[199]	training's rmse: 0.676095	valid_1's rmse: 0.782424
[200]	training's rmse: 0.675878	valid_1's rmse: 0.782396
[201]	training's rmse: 0.675714	valid_1's rmse: 0.782445
[202]	training's rmse: 0.675166	valid_1's rmse: 0.782496
[203]	training's rmse: 0.67501	valid_1's rmse: 0.782484
[204]	training's rmse: 0.674881	valid_1's rmse: 0.782478
[205]	training's rmse: 0.674727	valid_1's rmse: 0.782528
[206]	training's rmse: 0.674459	valid_1's rmse: 0.782532
[207]	training's rmse: 0.673853	valid_1's rmse: 0.782433
[208]	training's rmse: 0.673728	valid_1's rmse: 0.782391
[209]	training's rmse: 0.673599	valid_1's rmse: 0.782455
[210]	training's rmse: 0.673393	valid_1's rmse: 0.782367
[211]	training's rmse: 0.67323	v

[74]	training's rmse: 0.721716	valid_1's rmse: 0.816056
[75]	training's rmse: 0.721072	valid_1's rmse: 0.815914
[76]	training's rmse: 0.720612	valid_1's rmse: 0.816242
[77]	training's rmse: 0.719823	valid_1's rmse: 0.816113
[78]	training's rmse: 0.719392	valid_1's rmse: 0.816059
[79]	training's rmse: 0.718836	valid_1's rmse: 0.816108
[80]	training's rmse: 0.71837	valid_1's rmse: 0.816088
[81]	training's rmse: 0.718013	valid_1's rmse: 0.815823
[82]	training's rmse: 0.717205	valid_1's rmse: 0.815714
[83]	training's rmse: 0.716859	valid_1's rmse: 0.815591
[84]	training's rmse: 0.716493	valid_1's rmse: 0.815848
[85]	training's rmse: 0.715787	valid_1's rmse: 0.815645
[86]	training's rmse: 0.715432	valid_1's rmse: 0.815727
[87]	training's rmse: 0.71505	valid_1's rmse: 0.815796
[88]	training's rmse: 0.714484	valid_1's rmse: 0.815817
[89]	training's rmse: 0.714141	valid_1's rmse: 0.815636
[90]	training's rmse: 0.713835	valid_1's rmse: 0.815408
[91]	training's rmse: 0.713495	valid_1's rmse: 0.8

In [46]:
t = res[29].copy()
t['item_cnt_month'] = 0
for key in res:
    t2 = res[key].copy()
    t['item_cnt_month'] = t['item_cnt_month'] + t2['item_cnt_month']
    print(key)
t['item_cnt_month'] = t['item_cnt_month'] / 5
t[['ID', 'item_cnt_month']].to_csv('submit_ensemble_average.csv', index=False)

29
30
31
32
33


Train single model

In [33]:
df_train_input = df_input[df_input['date_block_num']<33]
df_val_input = df_input[df_input['date_block_num']==33]
df_test_input = df_input[df_input['date_block_num']==34]

In [35]:
train_lgb = lgb.Dataset(df_train_input.drop(['item_cnt_month', 'ID'], axis='columns'), \
            label=df_train_input['item_cnt_month'])
val_lgb = lgb.Dataset(df_val_input.drop(['item_cnt_month', 'ID'], axis='columns'), \
            label=df_val_input['item_cnt_month'])

In [ ]:
# Load model instead of training
bst = lgb.Booster(model_file='final_model')

In [37]:
num_round = 1500
# param = {'num_leaves': 200, 'objective': 'mse', 'metric': 'root_mean_squared_error'}
param = {'lambda_l1': 0.0027195002541221284, \
  'lambda_l2': 0.006749781970807508, \
  'num_leaves': 118, \
  'feature_fraction': 0.9347938981568078, \
  'bagging_fraction': 0.5923021997008711, \
  'bagging_freq': 7, \
  'min_child_samples': 93,
  'objective': 'mse', 'metric': 'root_mean_squared_error'}
bst = lgb.train(param, \
                train_lgb, \
                num_round, \
                valid_sets=[train_lgb, val_lgb], \
                callbacks=[lgb.early_stopping(100)])

[1]	training's rmse: 1.17377	valid_1's rmse: 1.09486
Training until validation scores don't improve for 100 rounds
[2]	training's rmse: 1.1299	valid_1's rmse: 1.06033
[3]	training's rmse: 1.09237	valid_1's rmse: 1.03197
[4]	training's rmse: 1.0609	valid_1's rmse: 1.0082
[5]	training's rmse: 1.03518	valid_1's rmse: 0.988181
[6]	training's rmse: 1.01158	valid_1's rmse: 0.969877
[7]	training's rmse: 0.991774	valid_1's rmse: 0.95495
[8]	training's rmse: 0.974987	valid_1's rmse: 0.942162
[9]	training's rmse: 0.960636	valid_1's rmse: 0.932335
[10]	training's rmse: 0.948166	valid_1's rmse: 0.923086
[11]	training's rmse: 0.937365	valid_1's rmse: 0.917475
[12]	training's rmse: 0.927871	valid_1's rmse: 0.91167
[13]	training's rmse: 0.920117	valid_1's rmse: 0.906195
[14]	training's rmse: 0.912743	valid_1's rmse: 0.903807
[15]	training's rmse: 0.906079	valid_1's rmse: 0.900665
[16]	training's rmse: 0.900366	valid_1's rmse: 0.898345
[17]	training's rmse: 0.895625	valid_1's rmse: 0.895125
[18]	train

[146]	training's rmse: 0.797968	valid_1's rmse: 0.869778
[147]	training's rmse: 0.797789	valid_1's rmse: 0.869616
[148]	training's rmse: 0.797641	valid_1's rmse: 0.869584
[149]	training's rmse: 0.797469	valid_1's rmse: 0.869482
[150]	training's rmse: 0.7973	valid_1's rmse: 0.869946
[151]	training's rmse: 0.797048	valid_1's rmse: 0.869968
[152]	training's rmse: 0.796837	valid_1's rmse: 0.869906
[153]	training's rmse: 0.796695	valid_1's rmse: 0.869856
[154]	training's rmse: 0.796551	valid_1's rmse: 0.869653
[155]	training's rmse: 0.796388	valid_1's rmse: 0.869629
[156]	training's rmse: 0.796077	valid_1's rmse: 0.869622
[157]	training's rmse: 0.795925	valid_1's rmse: 0.869609
[158]	training's rmse: 0.795093	valid_1's rmse: 0.869678
[159]	training's rmse: 0.794757	valid_1's rmse: 0.869649
[160]	training's rmse: 0.794298	valid_1's rmse: 0.869519
[161]	training's rmse: 0.79402	valid_1's rmse: 0.869605
[162]	training's rmse: 0.793801	valid_1's rmse: 0.869578
[163]	training's rmse: 0.793659	va

[290]	training's rmse: 0.768728	valid_1's rmse: 0.868559
[291]	training's rmse: 0.768363	valid_1's rmse: 0.868471
[292]	training's rmse: 0.768287	valid_1's rmse: 0.86842
[293]	training's rmse: 0.768137	valid_1's rmse: 0.868405
[294]	training's rmse: 0.767868	valid_1's rmse: 0.868372
[295]	training's rmse: 0.767727	valid_1's rmse: 0.868264
[296]	training's rmse: 0.767595	valid_1's rmse: 0.868357
[297]	training's rmse: 0.767311	valid_1's rmse: 0.868363
[298]	training's rmse: 0.767199	valid_1's rmse: 0.868312
[299]	training's rmse: 0.766995	valid_1's rmse: 0.868277
[300]	training's rmse: 0.76682	valid_1's rmse: 0.86826
[301]	training's rmse: 0.766623	valid_1's rmse: 0.868216
[302]	training's rmse: 0.766284	valid_1's rmse: 0.868212
[303]	training's rmse: 0.766191	valid_1's rmse: 0.868141
[304]	training's rmse: 0.766108	valid_1's rmse: 0.868169
[305]	training's rmse: 0.765994	valid_1's rmse: 0.86824
[306]	training's rmse: 0.76562	valid_1's rmse: 0.868123
[307]	training's rmse: 0.765549	vali

In [40]:
pred_test = bst.predict(df_test_input.drop(['item_cnt_month', 'ID'], axis='columns').values)
sub = df_test_input[['ID']].copy()
sub['ID'] = sub['ID'].astype(np.int32)
sub['item_cnt_month'] = pred_test
# sub['item_cnt_month'] = sub['item_cnt_month']*1.45

In [41]:
sub[['ID', 'item_cnt_month']].to_csv('submit.csv', index=False)

In [42]:
sub['item_cnt_month'].sum()

63220.809278841305

In [44]:
bst.save_model('final_model')